In [7]:
!pip install streamlit pyngrok plotly statsmodels


In [8]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import statsmodels.api as sm

# -------------------------------
# Title and description
# -------------------------------
st.markdown("""
    <h1 style='text-align: center;
               background: linear-gradient(to right, #ff7e5f, #feb47b);
               -webkit-background-clip: text;
               color: transparent;
               font-size: 50px;'>
           ULTIMATE Global Climate Explorer
    </h1>
""", unsafe_allow_html=True)

# Subtitle with Cool Blue Text
st.markdown("""
    <h3 style='text-align: center;
               color: #0072ff;
               font-style: italic;'>
        Explore global climate data: CO₂ emissions, temperature trends, and more.
    </h3>
""", unsafe_allow_html=True)


# -------------------------------
# Load and preprocess data
# -------------------------------
@st.cache_data
def load_data():
    # Load CO₂ data
    co2_url = 'https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv'
    co2_df = pd.read_csv(co2_url)

    # Fill missing population and GDP values
    co2_df['population'] = co2_df.groupby('country')['population'].ffill()
    co2_df['gdp'] = co2_df.groupby('country')['gdp'].ffill()

    # Fill missing CO₂ and GHG values
    co2_emission_cols = [col for col in co2_df.columns if 'co2' in col or 'ghg' in col]
    for col in co2_emission_cols:
        co2_df[col] = co2_df[col].fillna(0)

    # Fill other numeric columns
    numerical_cols = co2_df.select_dtypes(include=np.number).columns.tolist()
    cols_to_ffill = [col for col in numerical_cols if col not in ['year', 'population', 'gdp'] + co2_emission_cols]
    for col in cols_to_ffill:
        co2_df[col] = co2_df.groupby('country')[col].ffill()
        co2_df[col] = co2_df.groupby('country')[col].bfill()

    # Convert year to datetime
    co2_df['year'] = pd.to_datetime(co2_df['year'], format='%Y')

    # Dummy temperature data for merging
    temp_data = {
        'country': ['Afghanistan', 'Afghanistan', 'Albania', 'Albania'],
        'year': [datetime(1750, 1, 1), datetime(1751, 1, 1), datetime(1750, 1, 1), datetime(1751, 1, 1)],
        'average_temperature': [15.0, 15.2, 12.0, 12.5]
    }
    temperature_df = pd.DataFrame(temp_data)

    # Merge CO₂ and temperature data
    merged_df = pd.merge(co2_df, temperature_df, on=['country', 'year'], how='outer')

    return merged_df

data = load_data()

# -------------------------------
# Sidebar controls
# -------------------------------
st.sidebar.header("Settings")
countries = sorted(data['country'].unique().tolist())
countries.insert(0, 'World')

selected_country = st.sidebar.selectbox("Select a Country", countries)
min_year = data['year'].min().year
max_year = data['year'].max().year

year_range = st.sidebar.slider("Select Year Range", min_year, max_year, (min_year, max_year))

# Filter data
filtered_data = data[
    (data['country'] == selected_country) &
    (data['year'].dt.year >= year_range[0]) &
    (data['year'].dt.year <= year_range[1])
]

# -------------------------------
# Analysis section
# -------------------------------
st.header(f"Climate Analysis for {selected_country}")

if not filtered_data.empty:
    # CO₂ over time
    st.subheader("CO₂ Emissions Over Time")
    fig_co2 = px.line(filtered_data, x='year', y='co2', title=f'CO₂ Emissions for {selected_country}')
    st.plotly_chart(fig_co2, use_container_width=True)

    # CO₂ vs GDP
    st.subheader("CO₂ vs GDP")
    gdp_data = filtered_data.dropna(subset=['co2', 'gdp'])
    if not gdp_data.empty:
        fig_gdp = px.scatter(gdp_data, x='gdp', y='co2', title=f'CO₂ vs GDP for {selected_country}')
        st.plotly_chart(fig_gdp, use_container_width=True)

        # Correlation and Regression
        st.subheader("Correlation and Regression")
        st.write("Correlation Matrix:")
        st.write(gdp_data[['co2', 'gdp']].corr())

        X = sm.add_constant(gdp_data['gdp'])
        y = gdp_data['co2']
        model = sm.OLS(y, X).fit()
        st.text(model.summary())
    else:
        st.write("No GDP data for this country in the selected period.")

    # Global time series decomposition (only for 'World')
    if selected_country == 'World':
        st.subheader("Time Series Decomposition")
        from statsmodels.tsa.seasonal import seasonal_decompose
        ts = filtered_data.set_index('year')['co2'].dropna()
        decomposition = seasonal_decompose(ts, model='additive', period=1)

        fig = make_subplots(rows=4, cols=1, subplot_titles=['Original', 'Trend', 'Seasonal', 'Residual'])
        fig.add_trace(go.Scatter(x=ts.index, y=decomposition.observed, name='Original'), row=1, col=1)
        fig.add_trace(go.Scatter(x=ts.index, y=decomposition.trend, name='Trend'), row=2, col=1)
        fig.add_trace(go.Scatter(x=ts.index, y=decomposition.seasonal, name='Seasonal'), row=3, col=1)
        fig.add_trace(go.Scatter(x=ts.index, y=decomposition.resid, name='Residual'), row=4, col=1)
        fig.update_layout(height=900, title='Global CO₂ Time Series Decomposition')
        st.plotly_chart(fig, use_container_width=True)
else:
    st.write("No data for this country and year range.")

# -------------------------------
# Global overview
# -------------------------------
st.header("Global Overview")
recent_year = data['year'].max()
recent_data = data[(data['year'] == recent_year) & (data['country'] != 'World')]

if not recent_data.empty:
    st.subheader(f"Top 20 CO₂ Emitters in {recent_year.year}")
    top20 = recent_data.sort_values('co2', ascending=False).head(20)
    fig_top20 = px.bar(top20, x='country', y='co2', title=f"Top 20 CO₂ Emitters in {recent_year.year}")
    st.plotly_chart(fig_top20, use_container_width=True)

    st.subheader(f"CO₂ Emissions Map {recent_year.year}")
    fig_map = px.choropleth(recent_data, locations="iso_code", color="co2",
                            hover_name="country", title=f"CO₂ Emissions {recent_year.year}",
                            color_continuous_scale=px.colors.sequential.Plasma)
    st.plotly_chart(fig_map, use_container_width=True)


Overwriting app.py


In [10]:
!ngrok config add-authtoken 32TBaW0daeIghPAn9337TQk7zsT_4Py9Yw2LxAmSN9iXmrqBR

# Start Streamlit on port 8501
from pyngrok import ngrok
ngrok.kill()  # Kill any old tunnels
public_url = ngrok.connect(8501)
print("Public URL:", public_url)

# Run the app in the background
!streamlit run app.py --server.port 8501 > logs.txt 2>&1 &

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://01c534309dc5.ngrok-free.app" -> "http://localhost:8501"
